In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

Building prefix dict from the default dictionary ...
[DEBUG] Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
[DEBUG] Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.815 seconds.
[DEBUG] Loading model cost 0.815 seconds.
Prefix dict has been built successfully.
[DEBUG] Prefix dict has been built successfully.


In [ ]:
#download model
!axel https://obs.baimianxiao.cn/share/obs/sankagenkeshi/G_1293000.pth -n 72

流式输出内容被截断，只能显示最后 5000 行内容。
[ 93%]  .......... .......... .......... .......... ..........  [28796.8KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28796.8KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28796.8KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28798.4KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28798.4KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28798.4KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28798.4KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28798.4KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28798.4KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28799.8KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28799.8KB/s]
[ 93%]  .......... .......... .......... .......... ..........  [28799.8KB/s]
[ 93%]  .......... .......... .......

In [ ]:
hps_mt = utils.get_hparams_from_file("./configs/vc.json")

In [ ]:
npcList = ['派蒙', '凯亚', '安柏', '丽莎', '琴', '香菱', '枫原万叶',
           '迪卢克', '温迪', '可莉', '早柚', '托马', '芭芭拉', '优菈',
           '云堇', '钟离', '魈', '凝光', '雷电将军', '北斗',
           '甘雨', '七七', '刻晴', '神里绫华', '戴因斯雷布', '雷泽',
           '神里绫人', '罗莎莉亚', '阿贝多', '八重神子', '宵宫',
           '荒泷一斗', '九条裟罗', '夜兰', '珊瑚宫心海', '五郎',
           '散兵', '女士', '达达利亚', '莫娜', '班尼特', '申鹤',
           '行秋', '烟绯', '久岐忍', '辛焱', '砂糖', '胡桃', '重云',
           '菲谢尔', '诺艾尔', '迪奥娜', '鹿野院平藏']

net_g_mt = SynthesizerTrn(
    len(symbols),
    hps_mt.data.filter_length // 2 + 1,
    hps_mt.train.segment_size // hps_mt.data.hop_length,
    n_speakers=hps_mt.data.n_speakers,
    **hps_mt.model).cuda()
_ = net_g_mt.eval()


_ = utils.load_checkpoint("./G_1293000.pth", net_g_mt, None)

In [ ]:
#clone vits code
!git clone --depth=1 -b vc https://github.com/Stardust-minus/vits vits
!cp -r /content/vits/* /content
!rm -rf /content/vits
#install axel
!apt update
!apt install axel -y
#install requirements
!pip install -r requirements.txt
#build monotonic
!cd /content/monotonic_align && python setup.py build_ext --inplace

Cloning into 'vits'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 51 (delta 6), reused 40 (delta 5), pack-reused 0
Unpacking objects: 100% (51/51), done.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_6

Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'monotonic_align.core' extension
creating build
creating build/temp.linux-x86_64-3.7
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c core.c -o build/temp.linux-x86_64-3.7/core.o
creating /content/monotonic_align/monotonic_align
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-funct

In [ ]:
import fairseq
import librosa
class HubertFeatureReader(object):
    def __init__(self, ckpt_path, layer, max_chunk=1600000):
        (
            model,
            cfg,
            task,
        ) = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
        self.model = model[0].eval().cuda()
        self.task = task
        self.layer = layer
        self.max_chunk = max_chunk

    def read_audio(self, path, ref_len=None):
        wav, sr = librosa.load(path, sr=self.task.cfg.sample_rate)
        if wav.ndim == 2:
            wav = wav.mean(-1)
        assert wav.ndim == 1, wav.ndim
        if ref_len is not None and abs(ref_len - len(wav)) > 160:
            logging.warning(f"ref {ref_len} != read {len(wav)} ({path})")
        return wav

    def get_feats(self, path, ref_len=None):
        x = self.read_audio(path, ref_len=ref_len)
        with torch.no_grad():
            x = torch.from_numpy(x).float().cuda()
            if self.task.cfg.normalize:
                x = F.layer_norm(x, x.shape)
            x = x.view(1, -1)

            feat = []
            for start in range(0, x.size(1), self.max_chunk):
                x_chunk = x[:, start : start + self.max_chunk]
                feat_chunk, _ = self.model.extract_features(
                    source=x_chunk,
                    padding_mask=None,
                    mask=False,
                    output_layer=self.layer,
                )
                feat.append(feat_chunk)
        return torch.cat(feat, 1).squeeze(0)

In [ ]:
#download hubert model
!axel https://obs.baimianxiao.cn/share/obs/sankagenkeshi/checkpoint_best_legacy_500.pt -n 64

流式输出内容被截断，只能显示最后 5000 行内容。
[ 81%]  .......... .......... .......... .......... ..........  [29953.5KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29953.5KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29955.2KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29955.2KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29955.2KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29955.2KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29955.2KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29959.6KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29959.6KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29959.6KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29959.6KB/s]
[ 81%]  .......... .......... .......... .......... ..........  [29959.6KB/s]
[ 81%]  .......... .......... .......

In [ ]:
reader = HubertFeatureReader(r"./checkpoint_best_legacy_500.pt", 8, 1600000)

In [ ]:
#下面所使用的必须是wav
#如果不是wav文件需要先转换过去
speaker = '神里绫华'
wav_path=r"./你的文件.wav"

t = reader.get_feats(wav_path).squeeze()
with torch.no_grad():
    x_tst_mt = t.unsqueeze(0)
    x_tst_mt_lengths = torch.LongTensor([t.size(0)]).cuda()
    sid_mt = torch.LongTensor([npcList.index(speaker)]).cuda()
    audio_mt = net_g_mt.infer(x_tst_mt, x_tst_mt_lengths, sid=sid_mt, noise_scale=.1, noise_scale_w=.1, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(wav_path, normalize=False))    #源语音
ipd.display(ipd.Audio(audio_mt, rate=hps_mt.data.sampling_rate, normalize=False))   #转换语音